In [8]:
from mido import MidiFile, MidiTrack, Message
import numpy as np
import mido

In [9]:
# https://mido.readthedocs.io/en/latest/backends/rtmidi.html
mido.get_output_names()

ModuleNotFoundError: No module named 'rtmidi'

In [ ]:

port = mido.open_output('Port Name')

########### PROCESS MIDI FILE #############
mid = MidiFile("allegroconspirito.mid")  # a Mozart piece
for msg in mid.play():
    port.send(msg)